In [0]:
pip install PyDrive

In [0]:
import os
import pandas as pd
import numpy as np
import shutil

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import keras.layers
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (96, 96, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
# Compiling the CNN
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
dataset = pd.read_csv('/content/drive/My Drive/DataSets/HCD_Data/train_labels.csv')

file_names = list(dataset['id'].values)
img_labels = list(dataset['label'].values)

print(file_names)


In [0]:
from PIL import Image
import os, os.path

val_image_list = []
path = "/content/drive/My Drive/DataSets/HCD_Data/validation"

for f in os.listdir(path):
  f = f.replace('.tif', '')
  val_image_list.append(f)
  
print(val_image_list)


In [0]:
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/DataSets/HCD_Data/train', target_size = (96, 96), batch_size = 32, class_mode = 'binary')
validation_set = train_datagen.flow_from_directory('/content/drive/My Drive/DataSets/HCD_Data/validation', target_size = (96, 96), batch_size = 32, class_mode = 'binary')
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/DataSets/HCD_Data/test', target_size = (96, 96), batch_size = 32, class_mode = 'binary')

In [0]:
classifier.fit_generator(training_set,
steps_per_epoch = 50,
epochs = 10,
validation_data = validation_set,
validation_steps = 50)

In [0]:
# Part 3 - Making new predictions
import numpy as np
from keras.preprocessing import image

count = 0
  
for x in range(613):
  test_image = image.load_img(image_list[x],target_size = (96, 96))
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  result = classifier.predict(test_image)
  training_set.class_indices
  if result[0][0] >= 0.5:
    prediction = 'Atleast one cancerous cell present'
    
  else:
    prediction = 'No cancerous cells present'
    count += 1
  #print(prediction)
  #print(result[0][0])
    
print("correct = " + str(count))
print('acc = ' + str(count/613))

In [0]:
from PIL import Image
import os, os.path

image_list = []
path = "/content/drive/My Drive/DataSets/HCD_Data/train/1/"

for f in os.listdir(path):
  #f = f.replace('.tif', '')
  image_list.append(os.path.join(path, f))
  
print(len(image_list))


In [0]:
#code to save a pickel file

from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(classifier, 'HCD_keras.pkl')

In [0]:
#split the training images into 2 subfolder depending on the label(0 and 1)
for f in range(1500):
  
  pos = np.where(dataset['id'] == image_list[f])
  pos = int(pos[0])
  #print(pos)
  
  current_img = file_names[pos]
  current_label = img_labels[pos]
  
  #print('current_img = ' + current_img)
  shutil.move('/content/drive/My Drive/DataSets/HCD_Data/train/1/' + str(current_img) + '.tif', '/content/drive/My Drive/DataSets/HCD_Data/train/' + str(current_label) + '/' + current_img + '.tif')

In [0]:
for f in range(500):
  
  pos = np.where(dataset['id'] == val_image_list[f])
  pos = int(pos[0])
  
  current_img = file_names[pos]
  current_label = img_labels[pos]
  
  shutil.move('/content/drive/My Drive/DataSets/HCD_Data/validation/' + str(current_img) + '.tif', '/content/drive/My Drive/DataSets/HCD_Data/validation/' + str(current_label) + '/' + current_img + '.tif')

In [0]:
#code to find a file
import os

def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
find('HCD_keras.pkl', '/content')

In [0]:
#convert any array values or something to strings
def to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]
  
#Find the row number where a picture with a certain name is located
x = np.where(dataset['id'] ==  'pic_name

x = int(x[0])

x